In [ ]:
 import pandas as pd

In [ ]:
 velocity_1 = pd.read_csv('/content/drive/MyDrive/MEng Dissertation/final/FinalModel/room1_data/3d_velocity')
 spherical_in1 = pd.read_csv('/content/drive/MyDrive/MEng Dissertation/final/FinalModel/room1_data/spherical_inlet_1.csv')
 spherical_out1 = pd.read_csv('/content/drive/MyDrive/MEng Dissertation/final/FinalModel/room1_data/spherical_to_outlet_1.csv')
 walls_1 = pd.read_csv('/content/drive/MyDrive/MEng Dissertation/final/FinalModel/room1_data/walls_1.csv')

In [ ]:
#  velocity_2 = pd.read_csv('/content/drive/MyDrive/MEng Dissertation/final/FinalModel/room2_data/velocity_3d.csv')
#  spherical_in2 = pd.read_csv('/content/drive/MyDrive/MEng Dissertation/final/FinalModel/room2_data/spherical_inlet_2.csv')
#  spherical_out2 = pd.read_csv('/content/drive/MyDrive/MEng Dissertation/final/FinalModel/room2_data/spherical_to_outlet_2.csv')
#  walls_2 = pd.read_csv('/content/drive/MyDrive/MEng Dissertation/final/FinalModel/room2_data/walls.csv')

In [ ]:
room1 = pd.concat([velocity_1, spherical_in1, spherical_out1, walls_1], axis=1)

In [ ]:
# room2 = pd.concat([velocity_2, spherical_in2, spherical_out2, walls_2], axis=1)


In [ ]:
room1.head()

,Unnamed: 0,r,theta,phi,r_to_inlet,theta_to_inlet,phi_to_inlet,r_to_outlet,theta_to_outlet,phi_to_outlet,ceiling_distance,floor_distance,right_distance,left_distance,front_distance,back_distance
0,0,1.000000e-08,1.570796,3.141593,0.287903,1.079853,0.616073,0.212152,1.925606,3.236757,1.637560,1.362440,0.000000,5.000000,1.980007,3.019993
1,1,1.750039e-04,2.993730,1.573778,0.288127,1.079436,0.616784,0.212310,1.925882,3.236838,1.637560,1.362440,0.003765,4.996235,1.980007,3.019993
2,2,1.747512e-04,2.998899,1.575596,0.287477,1.078748,0.619447,0.212179,1.926969,3.239094,1.643377,1.356623,0.003767,4.996233,1.969998,3.030002
3,3,1.000000e-08,1.570796,3.141593,0.287253,1.079164,0.618735,0.212021,1.926692,3.239011,1.643377,1.356623,0.000000,5.000000,1.969998,3.030002
4,4,1.000000e-08,1.570796,3.141593,0.288097,1.077596,0.613406,0.212101,1.926829,3.234506,1.643364,1.356636,0.000000,5.000000,1.989999,3.010001


In [ ]:
# data = pd.concat([room1, room2])

In [ ]:
def categorize_regime(r):
    if r <= 0.0002:
        return 'stagnant'
    elif r <= 0.01:
        return 'laminar'
    elif r <= 0.021:
        return 'transition'
    else:
        return 'turbulent'

# Apply the function to create the new 'regime' column
room1['regime'] = room1['r'].apply(categorize_regime)


In [ ]:
room1.drop(['Unnamed: 0', 'r', 'theta', 'phi'], axis=1, inplace=True)

In [ ]:
room1.head()

,r_to_inlet,theta_to_inlet,phi_to_inlet,r_to_outlet,theta_to_outlet,phi_to_outlet,ceiling_distance,floor_distance,right_distance,left_distance,front_distance,back_distance,regime
0,0.287903,1.079853,0.616073,0.212152,1.925606,3.236757,1.637560,1.362440,0.000000,5.000000,1.980007,3.019993,stagnant
1,0.288127,1.079436,0.616784,0.212310,1.925882,3.236838,1.637560,1.362440,0.003765,4.996235,1.980007,3.019993,stagnant
2,0.287477,1.078748,0.619447,0.212179,1.926969,3.239094,1.643377,1.356623,0.003767,4.996233,1.969998,3.030002,stagnant
3,0.287253,1.079164,0.618735,0.212021,1.926692,3.239011,1.643377,1.356623,0.000000,5.000000,1.969998,3.030002,stagnant
4,0.288097,1.077596,0.613406,0.212101,1.926829,3.234506,1.643364,1.356636,0.000000,5.000000,1.989999,3.010001,stagnant


In [ ]:
y = room1[['regime']]

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder

In [ ]:
y.head()

,regime
0,stagnant
1,stagnant
2,stagnant
3,stagnant
4,stagnant


In [ ]:

label_encoder = LabelEncoder()
y['label'] = label_encoder.fit_transform(y['regime'])


<ipython-input-14-0e671dab9bc1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['label'] = label_encoder.fit_transform(y['regime'])


In [ ]:
room1.drop('regime', axis=1, inplace=True)

In [ ]:
label_mapping = dict(zip(label_encoder.classes_, range(4)))


In [ ]:
print("Label Mapping:")
for original_label, new_label in label_mapping.items():
    print(f"{original_label} -> {new_label}")

Label Mapping:
laminar -> 0
stagnant -> 1
transition -> 2
turbulent -> 3


In [ ]:
y_cat = keras.utils.to_categorical(y['label'], num_classes=4)

In [ ]:
y_cat = pd.DataFrame(y_cat)

In [ ]:
y_cat.head()

,0,1,2,3
0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0


In [ ]:

from tensorflow.keras import regularizers

# Build the ANN model with regularization and dropout
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(12,)),

    layers.Dense(128, activation='relu'),

    layers.Dense(256, activation='relu'),

    layers.Dense(512, activation='relu'),
    layers.Dense(4, activation='softmax')
])

# Compile the model with a different optimizer and learning rate
optimizer = keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with a different batch size and early stopping

model.fit(room1, y_cat, epochs=50, batch_size=100000, validation_split=0.2)



Epoch 1/50
274/274 [==============================] - 72s 237ms/step - loss: 0.3142 - accuracy: 0.8830 - val_loss: 0.2125 - val_accuracy: 0.9291
Epoch 2/50
274/274 [==============================] - 63s 229ms/step - loss: 0.2017 - accuracy: 0.9208 - val_loss: 0.2136 - val_accuracy: 0.9265
Epoch 3/50
274/274 [==============================] - 59s 217ms/step - loss: 0.1624 - accuracy: 0.9365 - val_loss: 0.1821 - val_accuracy: 0.9473
Epoch 4/50
274/274 [==============================] - 59s 218ms/step - loss: 0.1415 - accuracy: 0.9433 - val_loss: 0.1800 - val_accuracy: 0.9511
Epoch 5/50
274/274 [==============================] - 59s 215ms/step - loss: 0.1262 - accuracy: 0.9488 - val_loss: 0.1962 - val_accuracy: 0.9496
Epoch 6/50
274/274 [==============================] - 59s 217ms/step - loss: 0.1176 - accuracy: 0.9519 - val_loss: 0.1932 - val_accuracy: 0.9504
Epoch 7/50
274/274 [==============================] - 59s 217ms/step - loss: 0.1094 - accuracy: 0.9550 - val_loss: 0.1737 - val_ac

In [ ]:
batch_size = 100000  # Choose your desired batch size
predictions = model.predict(room1, batch_size=batch_size)


343/343 [==============================] - 35s 100ms/step


In [ ]:
true_class_labels = np.argmax(y_cat, axis=1)

# Convert predicted probabilities to class labels
predicted_class_labels = np.argmax(predictions, axis=1)

# Calculate accuracy for each class
class_accuracies = {}
for class_label in range(predictions.shape[1]):  # Assuming 4 classes
    # Filter predicted and true labels for the current class
    class_indices = np.where(true_class_labels == class_label)
    class_predicted = predicted_class_labels[class_indices]
    class_true = true_class_labels[class_indices]

    # Calculate accuracy for the current class
    class_accuracy = accuracy_score(class_true, class_predicted)
    class_accuracies[class_label] = class_accuracy

# Print the accuracy for each class
for class_label, accuracy in class_accuracies.items():
    print(f"Accuracy for class {class_label}: {accuracy:.4f}")

Accuracy for class 0: 0.9865
Accuracy for class 1: 0.8189
Accuracy for class 2: 0.9500
Accuracy for class 3: 0.9913


In [ ]:
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy for each class
class_accuracies = {}
for class_label in range(4):  # Assuming 'num_classes' is the total number of classes
    # Filter predicted and true labels for the current class
    class_indices = np.where(y_cat == class_label)
    class_predicted = predicted_labels[class_indices]
    class_true = y_cat[class_indices]

    # Calculate accuracy for the current class
    class_accuracy = accuracy_score(class_true, class_predicted)
    class_accuracies[class_label] = class_accuracy

# Print the accuracy for each class
for class_label, accuracy in class_accuracies.items():
    print(f"Accuracy for class {class_label}: {accuracy:.4f}")

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
class_accuracies = {}

# Loop through each class
for class_label in range(4):  # Assuming num_classes is the total number of classes
    # Find indices where true class labels match the current class
    indices = np.where(y_cat == class_label)[0]

    # Extract predicted class labels corresponding to the current class
    predicted_class_labels = predictions[indices]

    # Calculate accuracy for the current class
    accuracy = np.mean(predicted_class_labels == class_label)

    # Store accuracy in the dictionary
    class_accuracies[class_label] = accuracy

# Print accuracies for each class
for class_label, accuracy in class_accuracies.items():
    print(f'Accuracy for class {class_label}: {accuracy}')

Accuracy for class 0: 0.26967184171850483
Accuracy for class 1: 0.10188167456048339
Accuracy for class 2: nan
Accuracy for class 3: nan


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
                                                                                                                                                                                                                                               om sklearn.metrics import accuracy_score
import numpy as np

# Assuming 'predictions' are the predicted class labels
# Assuming 'y_cat' are the true class labels in one-hot encoded format

# Convert one-hot encoded labels to class labels
true_labels = np.argmax(y_cat, axis=1)

# Calculate accuracy for each class
class_accuracies = {}
for class_label in range(4):  # Assuming 'num_classes' is the total number of classes
    # Filter predicted and true labels for the current class
    class_indices = np.where(true_labels == class_label)[0]  # Get indices using [0]
    class_predicted = predictions[class_indices]
    class_true = true_labels[class_indices]

    # Check if there are any instances of the current class
    if len(class_indices) > 0:
        # Calculate accuracy for the current class
        class_accuracy = accuracy_score(class_true, class_predicted)
    else:
        # If there are no instances of the current class, set accuracy to 0
        class_accuracy = 0.0

    class_accuracies[class_label] = class_accuracy

# Print the accuracy for each class
for class_label, accuracy in class_accuracies.items():
    print(f"Accuracy for class {class_label}: {accuracy:.4f}")


ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets

In [ ]:
from sklearn.metrics import accuracy_score

# Assuming 'predicted_labels' are the predicted class labels
# Assuming 'true_labels' are the true class labels in one-hot encoded format

# Convert one-hot encoded labels to class labels
true_labels = np.argmax(y_cat, axis=1)

# Calculate accuracy for each class
class_accuracies = {}
for class_label in range(4):  # Assuming 'num_classes' is the total number of classes
    # Filter predicted and true labels for the current class
    class_indices = np.where(true_labels == class_label)
    class_predicted = predictions[class_indices]
    class_true = true_labels[class_indices]

    # Calculate accuracy for the current class
    class_accuracy = accuracy_score(class_true, class_predicted)
    class_accuracies[class_label] = class_accuracy

# Print the accuracy for each class
for class_label, accuracy in class_accuracies.items():
    print(f"Accuracy for class {class_label}: {accuracy:.4f}")


ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets

In [ ]:
true_labels = np.argmax(y_cat, axis=1)

# Calculate accuracy
accuracy = np.mean(predicted_labels == true_labels)

# Print the accuracy
print("Accuracy:", accuracy)

NameError: name 'predicted_labels' is not defined